In [20]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long)

x_user = torch.tensor([[1], [2]], dtype=torch.float)
x_record = torch.tensor([[1], [2]], dtype=torch.float)
x = torch.cat([x_user, x_record], dim=0)

data = Data(x=x, edge_index=edge_index.t().contiguous())
print(data)

Data(x=[4, 1], edge_index=[2, 2])


In [22]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [62]:
model = GCN(in_channels=1, out_channels=2)  # 입력 채널과 출력 채널 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.edge_index)
    loss.backward()
    optimizer.step()

TypeError: __init__() got an unexpected keyword argument 'in_channels'

In [13]:
model.eval()
_, pred = model(data).max(dim=1)
print(pred)

tensor([0, 0, 0, 0])


### GNN Recommendation

In [63]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# CSV 파일 불러오기
all_reviews = pd.read_csv('all_reviews.csv')

category_name = [ "", "영화", "뮤지컬", "연극", "스포츠", "공연", "드라마", "책", "전시"]

all_reviews['category_name'] = all_reviews['category'].apply(lambda x: category_name[x])
# 제목과 내용을 결합하여 하나의 텍스트로 생성
all_reviews['all_text'] = all_reviews['category_name'] + ' ' + all_reviews['title'] + ' ' + all_reviews['content']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_reviews['all_text'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [64]:
import torch
from torch_geometric.data import Data

# 유사도를 기반으로 엣지 생성 (유사도가 일정 수준 이상인 것만 연결)
edge_index = torch.tensor([
    [i, j] for i in range(len(cosine_sim)) for j in range(len(cosine_sim)) if cosine_sim[i, j] > 0.5
], dtype=torch.long).t().contiguous()

# TF-IDF 벡터를 노드 특징으로 사용
x = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float)

# 그래프 데이터 객체 생성
data = Data(x=x, edge_index=edge_index)
print(data.x.shape)

torch.Size([3417, 500])


In [65]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(500, 16)
        self.conv2 = GCNConv(16, 500)  # 최종 출력 차원을 원래 입력 차원으로 맞춤

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# 모델 학습 설정
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 모델 학습 과정
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.x)  # 차원이 맞춰진 후의 MSE Loss
    loss.backward()
    optimizer.step()

# 학습 반복
for epoch in range(100):
    train()

In [66]:
# from flask import Flask, request, jsonify
import numpy as np

#app = Flask(__name__)

# API로 유사한 기록 추천
# @app.route('/recommend', methods=['POST'])
def recommend():
    #user_data = request.json['user_text']  # 사용자가 쓴 텍스트 입력
    user_data = {
        'title': '엘지 우승가자',
        'content': '이영빈 선수 백투백 홈런 인생경기다!',
        'category': '스포츠'
    }
    user_data = user_data['category'] + " " + user_data['title'] + " " + user_data['content']
    user_vector = tfidf_vectorizer.transform([user_data])

    # 코사인 유사도 계산
    user_sim = cosine_similarity(user_vector, tfidf_matrix)

    # 유사도 순으로 추천 (상위 5개 추천)
    recommended_indices = np.argsort(-user_sim[0])
    recommendations = all_reviews.iloc[recommended_indices]

    return recommendations.to_dict(orient='records')

#if __name__ == '__main__':
#    app.run(debug=True)

In [67]:
recommend()

[{'title': '여름에 시원한 야구장',
  'content': '쾌적하고 좋네요 시원합니다',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 여름에 시원한 야구장 쾌적하고 좋네요 시원합니다'},
 {'title': '최강두산..ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ',
  'content': 'ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 최강두산..ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ'},
 {'title': '최강두산 승리고고',
  'content': 'ggggggggg',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 최강두산 승리고고 ggggggggg'},
 {'title': '대역전승 ㅜㅜ',
  'content': '너무짜릿했다...',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 대역전승 ㅜㅜ 너무짜릿했다...'},
 {'title': '두산최고',
  'content': 'ㅎㅎㅎㅎㅎㅎㅎ',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 두산최고 ㅎㅎㅎㅎㅎㅎㅎ'},
 {'title': '이겼다리 ㅎㅎㅎ',
  'content': 'ㄴㅇㅁㅀㅈㄷㄺㄴㅇㅁㄹㅇㅌㅊㄴㅍㅊㅌ',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 이겼다리 ㅎㅎㅎ ㄴㅇㅁㅀㅈㄷㄺㄴㅇㅁㄹㅇㅌㅊㄴㅍㅊㅌ'},
 {'title': '굿',
  'content': '재밌어용ㅎ',
  'category': 4,
  'category_name': '스포츠',
  'all_text': '스포츠 굿 재밌어용ㅎ'},
 {'title': 'lck',
  'content': '더 넓

### Connect to DB to Recommendation

In [9]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 사용
db_host = os.getenv("DATABASE_HOST")
db_name = os.getenv("DATABASE_NAME")
db_user = os.getenv("DATABASE_USER")
db_password = os.getenv("DATABASE_PASSWORD")

In [10]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime
from datetime import datetime

class Base(DeclarativeBase):
    pass
class User(Base):
    __tablename__ = "User"
    id = Column(Integer, primary_key=True)
    email = Column(String, unique=True)
    password = Column(String)

    def __repr__(self) -> str:
        return f"<User {self.id} {self.email}>"

class Profile(Base):
    __tablename__ = "Profile"
    id = Column(Integer, primary_key=True)
    userId = Column(Integer, ForeignKey("User.id"), nullable=False)
    nickname = Column(String, nullable=False)
    profilePhoto = Column(String, nullable=False)
    bio = Column(String, nullable=True)
    birthdate = Column(DateTime, nullable=True)

    def __repr__(self) -> str:
        return f"<Profile {self.id} {self.nickname} {self.userId}>"

class CulturePost(Base):
    __tablename__ = "CulturePost"
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    emoji = Column(String, nullable=False)
    date = Column(DateTime, nullable=False)
    categoryId = Column(Integer, nullable=False)
    authorId = Column(Integer, ForeignKey("User.id"), nullable=False)
    review = Column(String, nullable=False)
    disclosure = Column (String, nullable=False)
    detail1 = Column (String, nullable=True)
    detail2 = Column (String, nullable=True)
    detail3 = Column (String, nullable=True)
    detail4 = Column (String, nullable=True)
    createdAt = Column(DateTime, insert_default=datetime.now())
    updatedAt = Column(DateTime, default=datetime.now())

    def __repr__(self) -> str:
        return f"<CulturePost {self.id} {self.title} {self.emoji} {self.review}>"

class Photo(Base):
    __tablename__ = "Photo"
    id = Column(Integer, primary_key=True)
    url = Column(String, nullable=False)
    culturePostId = Column(Integer, ForeignKey("CulturePost.id"), nullable=False)

    def __repr__(self) -> str:
        return f"<Photo {self.url} {self.culturePostId}>"

In [11]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

# 데이터베이스 연결 설정
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}')

# 메타데이터 객체 생성
metadata = MetaData()

# 세션 생성기 설정
Session = sessionmaker(bind=engine)
session = Session()

# Fetch data from the CulturePost table
def fetch_data_from_db():
    query = session.query(CulturePost).all()
    data = [{'title': row.title, 'content': row.review, 'category': row.categoryId} for row in query]
    return data

# Fetch all records written by the current user
def fetch_user_data_from_db(user_id):
    query = session.query(CulturePost).filter_by(authorId=user_id).all()
    user_data = [{'title': row.title, 'content': row.review, 'category': row.categoryId} for row in query]
    return user_data

# Fetch the data
data_from_db = fetch_data_from_db()

In [12]:
import pandas as pd

# Convert fetched data into a pandas DataFrame
df = pd.DataFrame(data_from_db)

# Mapping category IDs to category names (similar to the previous example)
category_name = [ "", "영화", "뮤지컬", "연극", "스포츠", "공연", "드라마", "책", "전시"]
df['category_name'] = df['category'].apply(lambda x: category_name[x])

# Combine the category, title, and content into one field for vectorization
df['all_text'] = df['category_name'] + ' ' + df['title'] + ' ' + df['content']

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['all_text'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
def create_user_profile_vector(user_id):
    # Fetch all posts by the user
    user_posts = fetch_user_data_from_db(user_id)

    # Combine all posts into one large text
    combined_text = ' '.join([category_name[post['category']] + ' ' + post['title'] + ' ' + post['content'] for post in user_posts])
    print(combined_text)

    # Create a single TF-IDF vector for the combined text
    user_vector = tfidf_vectorizer.transform([combined_text])

    return user_vector

In [17]:
import numpy as np
def recommend(user_id):
    user_profile_vector = create_user_profile_vector(user_id)

    # Calculate similarity
    user_sim = cosine_similarity(user_profile_vector, tfidf_matrix)

    # Sort by similarity and recommend top 5
    recommended_indices = np.argsort(-user_sim[0])
    recommendations = df.iloc[recommended_indices]

    return recommendations.to_dict(orient='records')

In [24]:
ans = recommend(281)
print (len(ans))
print (ans[:5])

드라마 선재 업고 튀어 우리가 찾던 바로 그 청춘 로맨스물. 안 본 사람은 있어도 한번만 본 사람은 없는, 늦지않게 정주행해야 하는 작품. 연출진의 계산된 연출과 각본이 유치하지 않은 설렘으로 다가온다. 드라마 선재 업고 튀어 요즘 선재업고 튀어에 빠져있습니다. 그 이유는 드라마 슬기로운 의사생활 시즌 1 따듯하고 편안한 일상물. 영화 범죄도시4 전작의 시퀀스를 그대로 따라가지만 챙길 것 다 챙기는 영악한 구성.
1045
[{'title': '선재 업고 튀어', 'content': '우리가 찾던 바로 그 청춘 로맨스물. 안 본 사람은 있어도 한번만 본 사람은 없는, 늦지않게 정주행해야 하는 작품. 연출진의 계산된 연출과 각본이 유치하지 않은 설렘으로 다가온다.', 'category': 6, 'category_name': '드라마', 'all_text': '드라마 선재 업고 튀어 우리가 찾던 바로 그 청춘 로맨스물. 안 본 사람은 있어도 한번만 본 사람은 없는, 늦지않게 정주행해야 하는 작품. 연출진의 계산된 연출과 각본이 유치하지 않은 설렘으로 다가온다.'}, {'title': '선재 업고 튀어', 'content': '안 본 사람은 있어도 한 번만 본 사람은 없다. 매화 예측불가 엔딩,극본,연출,연기,ost까지 뭐하나 빠지지 않는 드라마♡ 풋풋했던 그 시절 청춘, 뜨거웠던 사랑이 그리운 분들께', 'category': 6, 'category_name': '드라마', 'all_text': '드라마 선재 업고 튀어 안 본 사람은 있어도 한 번만 본 사람은 없다. 매화 예측불가 엔딩,극본,연출,연기,ost까지 뭐하나 빠지지 않는 드라마♡ 풋풋했던 그 시절 청춘, 뜨거웠던 사랑이 그리운 분들께'}, {'title': '선재 업고 튀어', 'content': '똑똑한 드라마', 'category': 6, 'category_name': '드라마', 'all_text': '드라마 선재 업고 튀어 똑똑한 드라마'}, {'title': '선재 업고 튀어', 'content'

### GNN recommendation with DB

In [15]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 사용
db_host = os.getenv("DATABASE_HOST")
db_name = os.getenv("DATABASE_NAME")
db_user = os.getenv("DATABASE_USER")
db_password = os.getenv("DATABASE_PASSWORD")

In [16]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime
from datetime import datetime

class Base(DeclarativeBase):
    pass
class User(Base):
    __tablename__ = "User"
    id = Column(Integer, primary_key=True)
    email = Column(String, unique=True)
    password = Column(String)

    def __repr__(self) -> str:
        return f"<User {self.id} {self.email}>"

class Profile(Base):
    __tablename__ = "Profile"
    id = Column(Integer, primary_key=True)
    userId = Column(Integer, ForeignKey("User.id"), nullable=False)
    nickname = Column(String, nullable=False)
    profilePhoto = Column(String, nullable=False)
    bio = Column(String, nullable=True)
    birthdate = Column(DateTime, nullable=True)

    def __repr__(self) -> str:
        return f"<Profile {self.id} {self.nickname} {self.userId}>"

class CulturePost(Base):
    __tablename__ = "CulturePost"
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    emoji = Column(String, nullable=False)
    date = Column(DateTime, nullable=False)
    categoryId = Column(Integer, nullable=False)
    authorId = Column(Integer, ForeignKey("User.id"), nullable=False)
    review = Column(String, nullable=False)
    disclosure = Column (String, nullable=False)
    detail1 = Column (String, nullable=True)
    detail2 = Column (String, nullable=True)
    detail3 = Column (String, nullable=True)
    detail4 = Column (String, nullable=True)
    createdAt = Column(DateTime, insert_default=datetime.now())
    updatedAt = Column(DateTime, default=datetime.now())

    def __repr__(self) -> str:
        return f"<CulturePost {self.id} {self.title} {self.emoji} {self.review}>"

class Photo(Base):
    __tablename__ = "Photo"
    id = Column(Integer, primary_key=True)
    url = Column(String, nullable=False)
    culturePostId = Column(Integer, ForeignKey("CulturePost.id"), nullable=False)

    def __repr__(self) -> str:
        return f"<Photo {self.url} {self.culturePostId}>"

In [17]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

# 데이터베이스 연결 설정
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}')

# 메타데이터 객체 생성
metadata = MetaData()

# 세션 생성기 설정
Session = sessionmaker(bind=engine)
session = Session()

# Fetch data from the CulturePost table with all necessary columns
def fetch_data_from_db():
    query = session.query(CulturePost).all()
    data = [{
        'id': row.id,
        'title': row.title,
        'emoji': row.emoji,
        'date': row.date,
        'categoryId': row.categoryId,
        'authorId': row.authorId,
        'review': row.review,
        'disclosure': row.disclosure,
        'detail1': row.detail1,
        'detail2': row.detail2,
        'detail3': row.detail3,
        'detail4': row.detail4
    } for row in query]
    return data

# Fetch all records written by the current user with all necessary columns
def fetch_user_data_from_db(user_id):
    query = session.query(CulturePost).filter_by(authorId=user_id).all()
    user_data = [{
        'id': row.id,
        'title': row.title,
        'emoji': row.emoji,
        'date': row.date,
        'categoryId': row.categoryId,
        'authorId': row.authorId,
        'review': row.review,
        'disclosure': row.disclosure,
        'detail1': row.detail1,
        'detail2': row.detail2,
        'detail3': row.detail3,
        'detail4': row.detail4
    } for row in query]
    return user_data

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 준비 (DB에서 가져온 데이터를 바탕으로)
data_from_db = fetch_data_from_db()  # DB에서 가져온 콘텐츠 데이터
df = pd.DataFrame(data_from_db)
category_name = ["", "영화", "뮤지컬", "연극", "스포츠", "공연", "드라마", "책", "전시"]
df['category_name'] = df['categoryId'].apply(lambda x: category_name[x])
df['all_text'] = df['category_name'] + ' ' + df['title'] + ' ' + df['review']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['all_text'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [23]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import numpy as np

# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(500, 16)  # TF-IDF의 max_features = 500
        self.conv2 = GCNConv(16, 16)
        self.conv3 = GCNConv(16, 500)  # 차원 다시 확장

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        return x

# 유저 벡터와 전체 그래프 노드를 결합하여 학습 데이터 생성
def create_combined_graph_data(user_vector, data):
    # 유저 벡터의 크기를 맞춰 2차원으로 변환
    user_node = user_vector.squeeze(0)  # 불필요한 차원 제거

    # 유저 노드를 기존 TF-IDF 매트릭스의 한 노드로 추가
    combined_x = torch.cat([data.x, user_node.unsqueeze(0)], dim=0)

    # 엣지 구성 (유저 노드와 기존 노드 간 유사도 기반 엣지 추가)
    user_edges = [[i, len(data.x)] for i in range(len(data.x))]  # 유저 노드를 새로운 노드로 추가
    user_edge_index = torch.tensor(user_edges, dtype=torch.long).t().contiguous()

    combined_edge_index = torch.cat([data.edge_index, user_edge_index], dim=1)

    return Data(x=combined_x, edge_index=combined_edge_index)

def create_graph_data():
    # TF-IDF 벡터화된 데이터 사용 (이미 있는 tfidf_matrix와 cosine_sim 사용)
    # 엣지(Edge) 생성 (코사인 유사도에 기반)
    edge_index = torch.tensor([
        [i, j] for i in range(len(cosine_sim)) for j in range(len(cosine_sim)) if cosine_sim[i, j] > 0.5
    ], dtype=torch.long).t().contiguous()

    # TF-IDF 벡터를 노드 특징으로 사용
    x = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float)

    # 그래프 데이터 객체 생성
    data = Data(x=x, edge_index=edge_index)

    return data

def train_gnn(data):
    model = GCN()  # GCN 모델 생성
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # 옵티마이저 설정

    # 모델 학습 함수 정의
    def train():
        model.train()
        optimizer.zero_grad()
        out = model(data)  # GCN을 통해 노드 임베딩 계산
        loss = F.mse_loss(out, data.x)  # 노드 특징 복원을 위한 MSE Loss
        loss.backward()
        optimizer.step()

    # 학습 반복
    for epoch in range(100):  # 100회 반복 학습
        train()

    return model  # 학습 완료된 모델 반환

def create_user_profile_vector(user_id):
    # 유저가 작성한 모든 기록을 가져옴
    user_posts = fetch_user_data_from_db(user_id)

    # 카테고리, 제목, 내용을 결합하여 하나의 텍스트로 만듦
    combined_text = ' '.join([category_name[post['categoryId']] + ' ' + post['title'] + ' ' + post['review'] for post in user_posts])
    print(combined_text)
    
    # 결합된 텍스트를 TF-IDF 벡터화
    user_vector = tfidf_vectorizer.transform([combined_text])

    return torch.tensor(user_vector.toarray(), dtype=torch.float)

# 추천 함수 수정
def recommend(user_id):
    # 기존 데이터셋으로 그래프 생성
    data = create_graph_data()

    # GNN 학습
    trained_model = train_gnn(data)

    # 유저 프로필 벡터 생성
    user_vector = create_user_profile_vector(user_id)
    
    # 유저 벡터와 전체 노드 결합한 그래프 데이터 생성
    combined_data = create_combined_graph_data(user_vector, data)

    # GNN 모델로 유저 임베딩 추출
    trained_model.eval()  # 모델 평가 모드
    user_embedding = trained_model(combined_data)

    # 유저 임베딩과 모든 노드 간의 유사도 계산
    user_sim = cosine_similarity(user_embedding[-1].detach().numpy().reshape(1, -1), data.x.detach().numpy())

    # 유사한 순으로 정렬하여 추천
    recommended_indices = np.argsort(-user_sim[0])
    recommendations = df.iloc[recommended_indices]

    return recommendations.to_dict(orient='records')

In [24]:
ret = recommend(1)
print(len(ret))
print(ret[:5])

영화 파일럿 역시 믿고 보는 코미디 영화!
1045
[{'id': 1173, 'title': '인사이드 아웃 2', 'emoji': '📺', 'date': Timestamp('2024-08-26 00:00:00'), 'categoryId': 1, 'authorId': 393, 'review': 'Inside Out 2', 'disclosure': 'PUBLIC', 'detail1': '인사이드 아웃 2', 'detail2': None, 'detail3': '애니메이션, 가족', 'detail4': None, 'category_name': '영화', 'all_text': '영화 인사이드 아웃 2 Inside Out 2'}, {'id': 1110, 'title': '인사이드 아웃 2', 'emoji': '📺', 'date': Timestamp('2024-07-12 00:00:00'), 'categoryId': 1, 'authorId': 352, 'review': '픽사는 단순한 진리를 따듯한 영상미로 손쉽게 가슴으로 밀어 넣어주는 기술자들이다.', 'disclosure': 'PUBLIC', 'detail1': '인사이드 아웃 2', 'detail2': None, 'detail3': '애니메이션, 가족', 'detail4': None, 'category_name': '영화', 'all_text': '영화 인사이드 아웃 2 픽사는 단순한 진리를 따듯한 영상미로 손쉽게 가슴으로 밀어 넣어주는 기술자들이다.'}, {'id': 1100, 'title': '인사이드 아웃 2', 'emoji': '📺', 'date': Timestamp('2024-06-06 00:00:00'), 'categoryId': 1, 'authorId': 359, 'review': "남들이 생각하는 원래의 '나'란 무엇인가", 'disclosure': 'PUBLIC', 'detail1': '인사이드 아웃 2', 'detail2': None, 'detail3': '애니메이션, 가족', 'detail4':